In [1]:
import numpy as np
import pycosat
import itertools

In [2]:
def initialize(n):
    names = np.zeros([n,2*n,2*n],dtype=np.int)
    for i in range(n):
        for j in range(2*n):
            for k in range(2*n):
                names[i][j][k]= i*(2*n)**2 + j*2*n + k+1
    return names.tolist()

def encode_at_most_one(cnf, names, show):
    if show:
        print("enter function encode_at_most_one")
    n = len(names)
    #enc = []
    for i in range(n):
        for j in range(2*n):
            if show:
                print("in row ", i, ", column", j, ", there may only be at most 1 number, hence the following clauses:")
            for k in range(2*n):
                for l in range(1,(2*n)-k):
                    arr = [-1*names[i][j][k],-1*names[i][j][k+l]]
                    if show:
                        print(arr)
                    per_entry = cnf.insert(0,arr)
    return cnf

def encode_at_least_one(cnf,names,show):
    if show:
        print("enter function encode_at_least_one")
        print("for every field we expect a clause of", len(names[0]), "positive literals:")
    for i in names:
        atleast=[]
        for j in i:
            atleast.append(j)
            if show:
                print(j)
        cnf.extend(atleast)
    return cnf
    
def encode_exactly_one(cnf, names, show):
    if show:
        print("enter function encode_exactly_one")
    full = []
    full.extend(encode_at_most_one(cnf, names, show))
    full.extend(encode_at_least_one(cnf, names, show))
    return full



def encode_doku_as_cnf(names, show):
    cnf = []
    encode_exactly_one(cnf, names, show)
    if show:
        print("the doku is now translated to propositional logic in cnf format")
    return cnf

In [3]:
##### EXAMPLE
doku = initialize(2)
example = encode_doku_as_cnf(doku, True)
print( example )
pycosat.solve(example)

enter function encode_exactly_one
enter function encode_at_most_one
in row  0 , column 0 , there may only be at most 1 number, hence the following clauses:
[-1, -2]
[-1, -3]
[-1, -4]
[-2, -3]
[-2, -4]
[-3, -4]
in row  0 , column 1 , there may only be at most 1 number, hence the following clauses:
[-5, -6]
[-5, -7]
[-5, -8]
[-6, -7]
[-6, -8]
[-7, -8]
in row  0 , column 2 , there may only be at most 1 number, hence the following clauses:
[-9, -10]
[-9, -11]
[-9, -12]
[-10, -11]
[-10, -12]
[-11, -12]
in row  0 , column 3 , there may only be at most 1 number, hence the following clauses:
[-13, -14]
[-13, -15]
[-13, -16]
[-14, -15]
[-14, -16]
[-15, -16]
in row  1 , column 0 , there may only be at most 1 number, hence the following clauses:
[-17, -18]
[-17, -19]
[-17, -20]
[-18, -19]
[-18, -20]
[-19, -20]
in row  1 , column 1 , there may only be at most 1 number, hence the following clauses:
[-21, -22]
[-21, -23]
[-21, -24]
[-22, -23]
[-22, -24]
[-23, -24]
in row  1 , column 2 , there may on

[-1,
 -2,
 -3,
 4,
 -5,
 -6,
 -7,
 8,
 -9,
 -10,
 -11,
 12,
 -13,
 -14,
 -15,
 16,
 -17,
 -18,
 -19,
 20,
 -21,
 -22,
 -23,
 24,
 -25,
 -26,
 -27,
 28,
 -29,
 -30,
 -31,
 32]

In [4]:
a = initialize(2)

a=np.array(a)

a[:,::2].tolist()
print(a)

[[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]
  [13 14 15 16]]

 [[17 18 19 20]
  [21 22 23 24]
  [25 26 27 28]
  [29 30 31 32]]]


In [5]:
def getslice(k,names):
    cnf = []
    slice_C = []
    n = len(names)
    for i in range(n):
        for j in range(k,2*n,n):
            a = names[i][j]
            slice_C.append(a)
            
    return slice_C

def slice_cnf(names):
    cnf = []
    for i in range(n):
        sliced = getslice(i,names)
        atleast_n =[]
        comb_n = []
        for l in range(2*n):
            atleast= [k[l] for k in sliced]
            atleast_n.append(atleast)
            atleast_neg = [-x for x in atleast]
            comb = [list(z) for z in itertools.combinations(atleast_neg,n)]
            
           
            atleast_n.append(comb)
        cnf.append(atleast_n)
        #cnf.append(comb_n)
    return cnf

In [6]:
def encode_ring(cnf, names, show):
    if show:
        print("")
        print("enter encode_ring function")
        print("")
    n = len(names)
    for i in range(n):
        for k in range(2*n):
            at_least_clause = []
            if show:
                print("in row ", i, ", there may not be more than one", k+1)
            for j in range(2*n): 
                at_least_clause.append(names[i][j][k])
                at_most_clause = []
                for l in range(1,(2*n)-j):
                    at_most_clause = [-1*names[i][j][k], -1*names[i][j+l][k]]
                    cnf.extend([at_most_clause])
                    if show:
                        print(at_most_clause)
            cnf.extend([at_least_clause])
            if show:
                print("but there must be at least one ", k+1)
                print(at_least_clause)                
    return cnf

def encode_wedge(cnf, names, show):
    if show:
        print("")
        print("enter encode_wedge function")
        print("")
    n = len(names) # n is number of rings, but also number of wedges
    # first, collect all the variables that stand for a number "1" (or"k") in a wedge
    # then, construct clauses that say one of these variables must be true
    # but no more than one of thesevariables may be true
    for w in range(n): # per wedge
        for k in range(2*n): # per number
            wedge_content = []
            for i in range(n): # per row
                for j in range(2*w, 2*w+2): # per column
                    wedge_content.append(names[i][j][k])
            cnf.extend([wedge_content])
            if show:
                print("in wedge ", w, "there must be the number", k+1)
                print(wedge_content)
                print("but only one ", k+1)
            for l in range(2*n): # loop over the wedgecontent
                for m in range(1, 2*n - l):
                    clause = [-1*wedge_content[l], -1*wedge_content[l+m]]
                    if show:
                        print(clause)
    return cnf

def encode_givens(cnf, names, puzzle, show):
    n = len(names)
    for i in range(n):
        for j in range(2*n):
            for k in range(2*n):
                if puzzle[i,j] == k:
                    clause = [names[i][j][k]]
                    cnf.extend(clause)
                    if show:
                        print("givens-clause: ", clause)
    return cnf                   

In [7]:
a = np.loadtxt('valid_circledoku_2_rings_#0', delimiter=',', dtype='int' )

def reduce_CircleDoku(doku_puzzle):
    # this function takes a full, valid circledoku as input
    # and gives a proper circledoku with as few given as possible as output
    n = len(doku_puzzle)
    doku_indices = []
    for i in range(n):
        for j in range(2*n):
            doku_indices.append([i,j,doku_puzzle[i,j]])
    random.shuffle(doku_indices, random.random)
    print(doku_indices)
    
def indices_to_puzzle(doku_indices):
    n = len(doku_indices)/4
    doku_puzzle = np.zeros([n, 2*n], dtype=np.int)
    doku_puzzle.tolist()
    for i in range(len(doku_indices)):
        x = doku_indices[i][0]
        y = doku_indices[i][1]
        doku_puzzle[x][y] = doku_indices[i][2]
    return doku_puzzle
    
print(a)
reduce_CircleDoku(a)

FileNotFoundError: [Errno 2] No such file or directory: 'valid_circledoku_2_rings_#0'